In [1]:
import sys
sys.path.append('..')

from src.data.make_dataset import *
from tqdm import tqdm

In [2]:
embeddings_index = parse_glove_data(r"..\data\raw\glove.6B.100d.txt")

In [3]:
words_array = get_words_array(embeddings_index)
print("len(words_array) (raw):\t\t\t" + '{:,}'.format(len(words_array)))
words_array = filter_alphabetic(words_array)
print("len(words_array) (filter_alphabetic):\t" + '{:,}'.format(len(words_array)))
words_array = filter_20k(words_array, r'..\data\external\20k.txt')
print("len(words_array) (filter_20k):\t\t" + '{:,}'.format(len(words_array)))

len(words_array) (raw):			400,000
len(words_array) (filter_alphabetic):	317,756
len(words_array) (filter_20k):		19,737


In [4]:
np.random.choice(words_array, size=20, replace=False)

array(['sponsor', 'reptiles', 'analyzer', 'great', 'mpls',
       'authoritative', 'planning', 'charlotte', 'estimate', 'banquet',
       'native', 'sensations', 'breaking', 'cigarettes', 'product', 'wow',
       'examinations', 'diesel', 'aviation', 'colon'], dtype='<U39')

In [5]:
glove_df = embeddings_to_dataframe(embeddings_index, words_array)

In [6]:
starting_word = np.random.choice(words_array)
factor_words = np.random.choice(words_array, size=2, replace=False)
operations = np.random.choice([-1, 1], size=2)

factor_matrix = np.array([embeddings_index[word] for word in factor_words])
factors_x_operations = np.dot(operations, factor_matrix)

result_vec = embeddings_index[starting_word] + factors_x_operations
# final_words = list(glove_df.index[np.dot(glove_df, result_vec).argsort()[::-1][:5]])
final_words = glove_df.index[np.dot(glove_df, result_vec).argsort()[::-1][0]]

op_text = [" + " if o == 1 else " - " for o in list(operations)]
print(starting_word + op_text[0] + factor_words[0]
                    + op_text[1] + factor_words[1]
                    # + op_text[2] + factor_words[2]
                    + " = " + str(final_words))

vx + pogo - initiatives = vx


In [7]:
def simulate_game(words_array, glove_df, num_operations):
    starting_word = np.random.choice(words_array)
    factor_words = np.random.choice(words_array, size=num_operations, replace=False)
    operations = np.random.choice([-1, 1], size=num_operations)

    factor_matrix = np.array([embeddings_index[word] for word in factor_words])
    factors_x_operations = np.dot(operations, factor_matrix)

    result_vec = embeddings_index[starting_word] + factors_x_operations

    final_words = glove_df.index[np.dot(glove_df, result_vec).argsort()[::-1][:2]]
    final_word = final_words[0]
    if final_word == starting_word:
        final_word = final_words[1]
    if final_word in factor_words:
        final_word = final_words[1]

    similarity = np.dot(embeddings_index[final_word], result_vec)
    equation_string = starting_word
    for i in range(num_operations):
        if operations[i] == 1:
            equation_string += ' + '
        else:
            equation_string += ' - '
        equation_string += factor_words[i]
    equation_string += ' = ' + final_word

    return similarity, equation_string

simulate_game(words_array, glove_df, 2)

(14.875511996006523, 'dates - concluding - finite = cathedral')

In [9]:
sim_results = []
num_operations = 2
num_results = 10
threshold = 60

with tqdm(total=num_results, desc="Equations found") as pbar:
    while len(sim_results) < num_results:
        sim = simulate_game(words_array, glove_df, num_operations)
        if sim[0] > threshold:
            sim_results.append(sim)
            pbar.update(1)
results_df = pd.DataFrame(sim_results, columns=["similarity", "equation"])

sorted_results = results_df.sort_values(by="similarity", ascending=False)
sorted_results

Equations found: 100%|██████████| 10/10 [00:54<00:00,  5.46s/it]


,similarity,equation
6,74.561169,so + see + initial = we
1,69.107421,apple + ca + americans = you
4,66.720005,attorneys + we + task = they
2,65.638142,particularly + mm + elections = mm
0,64.334775,rates + new + straw = rate
8,63.490674,uranium + inspectors + gilmore = enrichment
3,62.410789,canoeing + canoeing + governed = kayaking
9,61.633886,things + pink + seven = you
5,61.469536,tcp + metacritic + pubmed = aggregator
7,60.220678,hazardous - ments + nuclear = weapons


In [11]:
np.dot(embeddings_index['king'] - embeddings_index['man'] + embeddings_index['woman'], embeddings_index['queen'])

29.906998